In [31]:
print('hi')

hi


In [2]:
import os
import scanpy as sc
import pandas as pd
import cellxgene_census

In [32]:
#input_dir = "/projects/bioinformatics/DB/scRNAseq_parkinson"
# the full path to the .h5ad file
#h5ad_path = os.path.join(input_dir, "dataset.h5ad")

In [33]:
 metadata = pd.read_csv('/projects/bioinformatics/DB/scRNAseq_parkinson/metadata.csv')

In [34]:
metadata.columns


Index(['barcodekey', 'n_genes', 'n_counts', 'Brain_bank', 'RIN',
       'path_braak_lb', 'derived_class2', 'PMI', 'tissue_ontology_term_id',
       'tissue_type', 'assay_ontology_term_id', 'disease_ontology_term_id',
       'cell_type_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id',
       'development_stage_ontology_term_id', 'sex_ontology_term_id',
       'donor_id', 'suspension_type', 'is_primary_data', 'cell_type', 'assay',
       'disease', 'sex', 'tissue', 'self_reported_ethnicity',
       'development_stage', 'observation_joinid'],
      dtype='object')

In [44]:
obs_join_ids = list(metadata['observation_joinid'])
obs_value_filter = 'observation_joinid in @obs_join_ids'

In [45]:
census = cellxgene_census.open_soma(census_version="2025-01-30")

# Meta Data 

In [47]:
cell_metadata = cellxgene_census.get_obs(
    census=census,
    organism="homo_sapiens",
    value_filter='dataset_id in ["d5d0df8f-4eee-49d8-a221-a288f50a1590"]',
    column_names=["cell_type", "assay", "donor_id", "development_stage", "suspension_type", "tissue_type"]
)


In [48]:
adata.obs_names

Index([], dtype='object')

In [53]:
# Some PBMC data from this data 
# 1. https://datasets.cellxgene.cziscience.com/88ad4023-93b2-4b70-a8a5-b0cf6c438cac.h5ad
#https://cellxgene.cziscience.com/collections/d5d0df8f-4eee-49d8-a221-a288f50a1590 <--- this we need to use

import cellxgene_census


emb_names = ["scvi", "geneformer"]


census_version = "2025-01-30"


#dataset_ids = ["d5d0df8f-4eee-49d8-a221-a288f50a1590"]


with cellxgene_census.open_soma(census_version=census_version) as census:
    adata = cellxgene_census.get_anndata(
        census,
        organism="homo_sapiens",
        measurement_name="RNA",
        obs_value_filter='dataset_id in ["d5d0df8f-4eee-49d8-a221-a288f50a1590"]',
        obs_embeddings=emb_names,
        obs_column_names = ["observation_joinid"]
    )


In [54]:
adata.obsm

AxisArrays with keys: scvi, geneformer

In [55]:
df_scvi = pd.DataFrame(
    adata.obsm["scvi"], 
    index=adata.obs_names
)
df_scvi.to_csv("scvi_embeddings.csv")

In [57]:
print(adata.obsm.keys())

KeysView(AxisArrays with keys: scvi, geneformer)


In [59]:
print(adata.shape)  
print(adata.obs.head())

(0, 61888)
Empty DataFrame
Columns: [observation_joinid, dataset_id]
Index: []


In [56]:
df_scvi

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49


In [60]:
df_geneformer = pd.DataFrame(
    adata.obsm["geneformer"], 
    index=adata.obs_names
)
df_geneformer.to_csv("geneformer_embeddings.csv")


In [61]:
df_geneformer 

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511


In [4]:
# Get the size in bytes
size_bytes = os.path.getsize(h5ad_path)

# Convert to a more readable unit (e.g., MB or GB)

size_gb = size_bytes / (1024 * 1024 * 1024)

print(f"Size in gigabytes: {size_gb:.2f} GB")

Size in gigabytes: 28.45 GB


In [6]:
import zarr

### Size issue 
That file size of 28.45 GB for a single-cell RNA-seq .h5ad dataset is very large and will likely pose significant computational challenges for a standard analysis,
especially when using a Transformer-based model like GenFormer.

Memory (RAM): A 28 GB .h5ad file, which typically contains a sparse matrix of UMI counts, will require a substantial amount of RAM when loaded and processed. When the data is converted to a dense matrix or intermediate processing steps are performed (like calculating nearest neighbors, running PCA, etc.), the memory footprint can easily exceed 100-200 GB. If you don't have access to a high-memory computing environment (e.g., a powerful server or cloud instance), the analysis will crash.

Processing Time: Even with sufficient RAM, a dataset of this size (likely involving millions of cells) will result in long processing times for any single-cell workflow, particularly for computationally intensive steps like training a large neural network model such as GenFormer.